# Language classification model using the XLM-RoBERTa transformer.

Load packages

In [29]:
import pandas as pd
from transformers import pipeline, XLMRobertaForSequenceClassification, XLMRobertaTokenizer, Trainer, TrainingArguments
from datasets import load_dataset, concatenate_datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

Load and Filter Dataset:

Load the dataset and filter it to include only selected languages (English, Spanish, French, German, and Italian). Take quick sample of the filtered dataset for prototyping.

In [30]:
# Step 1: Load and preprocess the dataset
# Load the dataset and filter for the target languages
df = pd.read_csv("sentences.csv", sep="\t", header=None, names=["id", "lang", "text"])
target_languages = ["eng", "spa", "fra", "deu", "ita"]
df_filtered = df[df['lang'].isin(target_languages)]

# Show select sample
df_sample = df_filtered.sample(1000)
print("Sample of the dataset:\n", df_sample.head())

KeyboardInterrupt: 

Prepare Data for Training: 

Extract texts and labels, then split the data into training and test sets.

In [3]:
# Step 2: Split data for training and testing
texts = df_sample['text'].tolist()
labels = df_sample['lang'].tolist()

# Split data into training and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

Load Model:

Load the XLM-RoBERTa model for text classification.

In [4]:
# Step 3: Load the pre-trained classification pipeline for initial testing
classifier = pipeline("text-classification", model="xlm-roberta-base")

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Make Predictions:

Predict the language on the test set and evaluate results.

In [5]:
# Step 4: Get initial predictions for the test set
predictions = classifier(test_texts)

# Extract predicted labels
predicted_labels = [prediction['label'] for prediction in predictions]

In [6]:
# Evaluate the initial performance
print("Initial Model Accuracy:", accuracy_score(test_labels, predicted_labels))
print("Classification Report:\n", classification_report(test_labels, predicted_labels))

Initial Model Accuracy: 0.0
Classification Report:
               precision    recall  f1-score   support

     LABEL_1       0.00      0.00      0.00       0.0
         deu       0.00      0.00      0.00      29.0
         eng       0.00      0.00      0.00      83.0
         fra       0.00      0.00      0.00      32.0
         ita       0.00      0.00      0.00      35.0
         spa       0.00      0.00      0.00      21.0

    accuracy                           0.00     200.0
   macro avg       0.00      0.00      0.00     200.0
weighted avg       0.00      0.00      0.00     200.0



/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/codespace/.loca

In [7]:
# Define the mapping from label to actual language
label_to_language = {
    'LABEL_0': 'English',
    'LABEL_1': 'Spanish',
    'LABEL_2': 'French',
    'LABEL_3': 'German',
    'LABEL_4': 'Italian'
}

# Step 5: Test the model with example sentences
# Predict the language of example sentences to verify the model's performance on known cases
example_sentences = [
    "This is a test sentence in English.",
    "Esta es una frase de prueba en español.",
    "C'est une phrase de test en français.",
    "Dies ist ein Testsatz auf Deutsch.",
    "Questa è una frase di prova in italiano."
]

# Predict the language and map the label to actual language name
print("Testing example sentences:")
for sentence in example_sentences:
    prediction = classifier(sentence)
    predicted_language = label_to_language[prediction[0]['label']]
    print(f"Sentence: {sentence}")
    print(f"Predicted Language: {predicted_language}\n")

Testing example sentences:
Sentence: This is a test sentence in English.
Predicted Language: Spanish

Sentence: Esta es una frase de prueba en español.
Predicted Language: Spanish

Sentence: C'est une phrase de test en français.
Predicted Language: Spanish

Sentence: Dies ist ein Testsatz auf Deutsch.
Predicted Language: Spanish

Sentence: Questa è una frase di prova in italiano.
Predicted Language: Spanish



Load and prepare multilingual training dataset:

Load dataset (PAWS-X) for training and testing the model, and apply tokenization.

In [8]:
# Step 6: Load PAWS-X for English, Spanish, German, French, and Italian
dataset_en = load_dataset('xtreme', 'PAWS-X.en')
dataset_es = load_dataset('xtreme', 'PAWS-X.es')
dataset_de = load_dataset('xtreme', 'PAWS-X.de')
dataset_fr = load_dataset('xtreme', 'PAWS-X.fr')
dataset_it = load_dataset('xtreme', 'PAN-X.it')

# Concatenate the datasets
train_dataset = concatenate_datasets([dataset_en['train'], dataset_es['train'], dataset_de['train'], dataset_fr['train'], dataset_it['train']])
test_dataset = concatenate_datasets([dataset_en['test'], dataset_es['test'], dataset_de['test'], dataset_fr['test'], dataset_it['test']])

Tokenize dataset

In [9]:
# Initialize the tokenizer
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

# Tokenization function
def tokenize(batch):
    # Concatenate sentence1 and sentence2, replacing None values with an empty string
    combined_sentences = [(s1 or "") + " " + (s2 or "") for s1, s2 in zip(batch['sentence1'], batch['sentence2'])]
    return tokenizer(combined_sentences, padding="max_length", truncation=True, max_length=128)

Apply tokenization to train and test datasets

In [10]:
train_data = train_dataset.map(tokenize, batched=True)
test_data = test_dataset.map(tokenize, batched=True)

Convert Labels to Integer Format

Filter out samples with None labels and convert labels to integers.

In [11]:
# Convert labels to integers and remove null labels
train_data = train_data.filter(lambda x: x['label'] is not None)
test_data = test_data.filter(lambda x: x['label'] is not None)

# Convert labels to integers for PyTorch compatibility
train_data = train_data.map(lambda x: {'label': int(x['label'])})
test_data = test_data.map(lambda x: {'label': int(x['label'])})

Set format for PyTorch:

Prepare the dataset for use with PyTorch.

In [12]:
# Set format for PyTorch
train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Now train_data and test_data are ready to be used with Trainer.

First, 
Set up the model and specify training arguments.

In [13]:
# Step 7: Fine-tune the model
model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=5)  # For the five languages

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Define Training Arguments

In [26]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="no",  # Optional: turn off intermediate evaluations
    learning_rate=5e-5,  # Increased slightly for faster convergence
    per_device_train_batch_size=32,  # Larger batch size
    per_device_eval_batch_size=32,   # Match eval batch size to train batch size
    num_train_epochs=0.5,  # Fewer epochs
    weight_decay=0.01,
    fp16=True,  # Enable mixed-precision training if supported
)

Create the Trainer:

Initialize the Trainer object.

In [27]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
)

Train and Evaluate Model: 

Finally, Train the model and evaluate its performance.

In [28]:
# Train the model
trainer.train()

Step,Training Loss
500,0.403200
1000,0.296700
1500,0.259000
2000,0.227600
2500,0.210100
3000,0.192000


TrainOutput(global_step=3084, training_loss=0.26261100292824274, metrics={'train_runtime': 14032.8817, 'train_samples_per_second': 7.032, 'train_steps_per_second': 0.22, 'total_flos': 6491650810871808.0, 'train_loss': 0.26261100292824274, 'epoch': 0.5})

Evaluate the Model

In [31]:
# Evaluate the model on the test set
trainer.evaluate()

{'eval_loss': 0.24888308346271515,
 'eval_runtime': 334.2388,
 'eval_samples_per_second': 23.935,
 'eval_steps_per_second': 0.748,
 'epoch': 0.5}

Save the Model:

In [32]:
# Save the model and tokenizer
model.save_pretrained("./language_predictor_model")
tokenizer.save_pretrained("./language_predictor_model")

('./language_predictor_model/tokenizer_config.json',
 './language_predictor_model/special_tokens_map.json',
 './language_predictor_model/sentencepiece.bpe.model',
 './language_predictor_model/added_tokens.json')

Test with New Sentences:

In [34]:
# Step 8: Load the Model and Tokenizer for Future Use
model = XLMRobertaForSequenceClassification.from_pretrained("./language_predictor_model")
tokenizer = XLMRobertaTokenizer.from_pretrained("./language_predictor_model")
language_predictor = pipeline("text-classification", model=model, tokenizer=tokenizer)

In [35]:
# Define the prediction function using the loaded model
def predict_language(sentences):
    for sentence in sentences:
        prediction = language_predictor(sentence)
        predicted_label = prediction[0]['label']
        predicted_language = label_to_language[predicted_label]
        
        print(f"Sentence: {sentence}")
        print(f"Predicted Language: {predicted_language}\n")

In [38]:
# Example sentences for prediction
sentences = [
    "This is an English sentence.",
    "Esta es una frase en español.",
]

predict_language(sentences)

Sentence: This is an English sentence.
Predicted Language: English

Sentence: Esta es una frase en español.
Predicted Language: Spanish

